In [ ]:
from tifffile import imread
from tifffile import imwrite
from tifffile import imsave
from matplotlib import pyplot
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import pickle
import itertools

In [ ]:
# import reference satellite images and check the data
rf_sl_20201204p = imread('file_name')
rf_sl_20210114p = imread('file_name')
rf_sl_20210122p = imread('file_name')
rf_sl_20210201p = imread('file_name')
rf_sl_20201204w = imread('file_name')
rf_sl_20210114w = imread('file_name')
rf_sl_20210122w = imread('file_name')
print('rf_20201204p shape:',rf_sl_20201204p.shape, 'dtype:',rf_sl_20201204p.dtype)
print('rf_20210114p shape:',rf_sl_20210114p.shape, 'dtype:',rf_sl_20210114p.dtype)
print('rf_20210122p shape:',rf_sl_20210122p.shape, 'dtype:',rf_sl_20210122p.dtype)
print('rf_20210201p shape:',rf_sl_20210201p.shape, 'dtype:',rf_sl_20210201p.dtype)
print('rf_20201204w shape:',rf_sl_20201204w.shape, 'dtype:',rf_sl_20201204w.dtype)
print('rf_20210114w shape:',rf_sl_20210114w.shape, 'dtype:',rf_sl_20210114w.dtype)
print('rf_20210122w shape:',rf_sl_20210122w.shape, 'dtype:',rf_sl_20210122w.dtype)

In [ ]:
# flatten the first two dimension, use * to iteratively represent all the elements after [2:] 
rf_sl_20201204p_fl = rf_sl_20201204p.reshape(-1, *rf_sl_20201204p.shape[2:]) 
rf_sl_20210114p_fl = rf_sl_20210114p.reshape(-1, *rf_sl_20210114p.shape[2:])
rf_sl_20210122p_fl = rf_sl_20210122p.reshape(-1, *rf_sl_20210122p.shape[2:])
rf_sl_20210201p_fl = rf_sl_20210201p.reshape(-1, *rf_sl_20210201p.shape[2:])
rf_sl_20201204w_fl = rf_sl_20201204w.reshape(-1, *rf_sl_20201204w.shape[2:])
rf_sl_20210114w_fl = rf_sl_20210114w.reshape(-1, *rf_sl_20210114w.shape[2:])
rf_sl_20210122w_fl = rf_sl_20210122w.reshape(-1, *rf_sl_20210122w.shape[2:])

rf_sl_2021 = np.concatenate([rf_sl_20201204p_fl,rf_sl_20210114p_fl,rf_sl_20210122p_fl,
                             rf_sl_20210201p_fl,rf_sl_20201204w_fl,rf_sl_20210114w_fl,
                             rf_sl_20210122w_fl])

rf_sl_2021 = rf_sl_2021[...,np.newaxis]

rf_sl_2021_nan = np.where(rf_sl_2021 == 0, np.nan, rf_sl_2021) # replace 0 with nan for later row dropping

print('rf_sl_shape_2021:',rf_sl_2021_nan.shape)
print('rf_sl_type_2021:',rf_sl_2021_nan.dtype)

In [ ]:
# import PlanetScope (PS) images and check the data
sl_20201204p = imread('file_name')
sl_20210114p = imread('file_name')
sl_20210122p = imread('file_name')
sl_20210201p = imread('file_name')
sl_20201204w = imread('file_name')
sl_20210114w = imread('file_name')
sl_20210122w = imread('file_name')
sl_20201204w = sl_20201204w[2:,:-2]
sl_20210114w = sl_20210114w[1:-1,1:-1]
sl_20210122w = sl_20210122w[1:,:-1]
print('20201204p shape:',sl_20201204p.shape, 'dtype:',sl_20201204p.dtype)
print('20210114p shape:',sl_20210114p.shape, 'dtype:',sl_20210114p.dtype)
print('20210122p shape:',sl_20210122p.shape, 'dtype:',sl_20210122p.dtype)
print('20210201p shape:',sl_20210201p.shape, 'dtype:',sl_20210201p.dtype)
print('20201204w shape:',sl_20201204w.shape, 'dtype:',sl_20201204w.dtype)
print('20210114w shape:',sl_20210114w.shape, 'dtype:',sl_20210114w.dtype)
print('20210122w shape:',sl_20210122w.shape, 'dtype:',sl_20210122w.dtype)

In [ ]:
# flatten the first two dimension
sl_20201204p_fl = sl_20201204p.reshape(-1, *sl_20201204p.shape[2:]) 
sl_20210114p_fl = sl_20210114p.reshape(-1, *sl_20210114p.shape[2:])
sl_20210122p_fl = sl_20210122p.reshape(-1, *sl_20210122p.shape[2:])
sl_20210201p_fl = sl_20210201p.reshape(-1, *sl_20210201p.shape[2:])
sl_20201204w_fl = sl_20201204w.reshape(-1, *sl_20201204w.shape[2:])
sl_20210114w_fl = sl_20210114w.reshape(-1, *sl_20210114w.shape[2:])
sl_20210122w_fl = sl_20210122w.reshape(-1, *sl_20210122w.shape[2:])

sl_2021 = np.concatenate([sl_20201204p_fl,sl_20210114p_fl,sl_20210122p_fl,
                          sl_20210201p_fl,sl_20201204w_fl,sl_20210114w_fl,
                          sl_20210122w_fl], axis=0)

sl_2021_nan = np.where(sl_2021 == 65535, np.nan, sl_2021).astype('float32') # replace 65535 with nan for later row dropping

print('sl_shape_2021:',sl_2021_nan.shape)
print('sl_type_2021:',sl_2021_nan.dtype)

In [ ]:
# concatenate the reference satellite data and PS data
sl_cc_2021 = np.concatenate([rf_sl_2021_nan,sl_2021_nan], axis=1)
print('sl_cc_shape_2021:',sl_cc_2021.shape)
print('sl_cc_type_2021:',sl_cc_2021.dtype)

In [ ]:
# make train_test array back to dataframe
df_sl_2021 = pd.DataFrame(sl_cc_2021)
df_sl_2021.columns = ['GWS','costal_B','B','G1','G2','Y','R','R_edge','NIR']

# drop rows with GWS column having nan
df_sl_clean_2021 = df_sl_2021.dropna(axis=0, how='any').reset_index(drop=True)
print('df_sl_clean_shape_2021:',df_sl_clean_2021.shape)
print('df_sl_clean_type_2021:',df_sl_clean_2021.dtypes)

In [ ]:
# compute NDSI based on all possible pair-wise combinations
df_NDSI_2021 = pd.DataFrame()
df_sl_X_2021 = df_sl_clean_2021.iloc[:,1:]
for a, b in itertools.combinations(df_sl_X_2021.columns, 2):
    df_NDSI_2021[f'{a}-{b}/{a}+{b}'] = (df_sl_X_2021[a]-df_sl_X_2021[b]).div(df_sl_X_2021[a]+df_sl_X_2021[b])
print('df_NDSI_shape_2021:',df_NDSI_2021.shape)
print('df_NDSI_type_2021:',df_NDSI_2021.dtypes)

In [ ]:
# choose an optimal random state number
test_list = []
for i in range(0, 43):
    X_2021, y_2021 = df_NDSI_2021.values, df_sl_clean_2021.iloc[:,0].values
    X_train_2021, X_test_2021, y_train_2021, y_test_2021 = train_test_split(X_2021, y_2021, test_size=0.3, 
                                    random_state=i)
    output = [i,np.mean(y_train_2021),np.mean(y_test_2021),np.std(y_train_2021),np.std(y_test_2021)]
    test_list.append(output)
df_test_list = pd.DataFrame(test_list, columns =['random state','train_mean','test_mean','train_std','test_std'])
df_test_list.to_csv('CSV_file')

In [ ]:
# train_test set splitting 
X_2021, y_2021 = df_NDSI_2021.values, df_sl_clean_2021.iloc[:,0].values
X_train_2021, X_test_2021, y_train_2021, y_test_2021 = train_test_split(X_2021, y_2021, test_size=0.3, 
                                    random_state=5)

In [ ]:
# check data 
print('any NaN in X_2021?', np.any(np.isnan(X_2021)))
print('any NaN in y_2021?', np.any(np.isnan(y_2021)))
print('any infinity in X_2021?', np.any(np.isinf(X_2021)))
print('any infinity in y_2021?', np.any(np.isinf(y_2021)))

In [ ]:
# standardization
scaler_st = StandardScaler()
X_train_2021_st = scaler_st.fit_transform(X_train_2021)
X_test_2021_st = scaler_st.transform(X_test_2021)

In [ ]:
# Multi-layer Perceptron 
mlp = MLPRegressor(random_state=0, max_iter=20000)
mlp_para = {'hidden_layer_sizes': [(28,),(28,14),(28,14,7),(28,14,7,3)],
            'activation': ['tanh', 'relu'], 'solver': ['sgd', 'adam'], 
            'learning_rate': ['constant','adaptive'],
            'alpha': [0.0001,0.001,0.01]}
mlp_gs = GridSearchCV(mlp, mlp_para, cv = 10, scoring='r2', n_jobs=8)
mlp_gs.fit(X_train_2021_st, y_train_2021)
mlp_train_r2 = mlp_gs.score(X_train_2021_st, y_train_2021)
mlp_test_r2_2021 = mlp_gs.score(X_test_2021_st, y_test_2021)
mlp_test_rmse_2021 = mean_squared_error(y_test_2021, 
                                        mlp_gs.predict(X_test_2021_st), 
                                      squared=False)
print(mlp_train_r2, mlp_test_r2_2021, mlp_test_rmse_2021)

In [ ]:
# Random forest regression
rf = RandomForestRegressor(max_depth=15, n_estimators=500, random_state=0, 
                           n_jobs=8)
rf_para = {'max_features':['auto', 'sqrt', 'log2']}
rf_gs = GridSearchCV(rf, rf_para, cv = 10, scoring='r2', n_jobs=8)
rf_gs.fit(X_train_2021_st, y_train_2021)
rf_train_r2 = rf_gs.score(X_train_2021_st, y_train_2021)
rf_test_r2_2021 = rf_gs.score(X_test_2021_st, y_test_2021)
rf_test_rmse_2021 = mean_squared_error(y_test_2021, 
                                       rf_gs.predict(X_test_2021_st), 
                                      squared=False)
print(rf_train_r2, rf_test_r2_2021, rf_test_rmse_2021)

In [ ]:
# Support vector regression
svr = SVR()
svr_para = {'kernel':['linear','poly','rbf'], 'C':[0.01,0.1,1,10,100], 
                        'gamma':['scale','auto'], 'epsilon':[0.1,0.5,0.9]}
svr_gs = GridSearchCV(svr, svr_para, cv = 10, scoring='r2', n_jobs=8)
svr_gs.fit(X_train_2021_st, y_train_2021)
svr_train_r2 = svr_gs.score(X_train_2021_st, y_train_2021)
svr_test_r2_2021 = svr_gs.score(X_test_2021_st, y_test_2021)
svr_test_rmse_2021 = mean_squared_error(y_test_2021, 
                                        svr_gs.predict(X_test_2021_st),
                                       squared=False)
print(svr_train_r2, svr_test_r2_2021, svr_test_rmse_2021)

In [ ]:
# save trained model
pickle.dump(rf_gs, open('file_name', 'wb'))

In [ ]:
#make regression graph
graph_x1 = y_train_2021 
graph_y1 = rf_gs.predict(X_train_2021_st)
graph_x2 = y_test_2021
graph_y2 = rf_gs.predict(X_test_2021_st)
pyplot.figure(dpi=600)
pyplot.plot([200,1200], [200,1200], 'r:')
scatter1 = pyplot.scatter(graph_x1, graph_y1, s=0.05)
scatter2 = pyplot.scatter(graph_x2, graph_y2, s=0.05)
lgnd = pyplot.legend((scatter1,scatter2), ('training set','test set',), loc='lower right')
lgnd.legendHandles[0]._sizes = [5] # point size in legend
lgnd.legendHandles[1]._sizes = [5]
pyplot.ylabel('predicted Ѱstem (kPa)')
pyplot.xlabel('reference Ѱstem (kPa) from 3m calibrated UAV images')
pyplot.axis([200,1200, 200,1200])
pyplot.savefig('plot_file',dpi=600)
pyplot.show()